# An array of sequences

*Example 2-4. Cartesian product using a list comprehension.*

In [4]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts 

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

## Generator expressions
*Example 2-5. Initializing a tuple and an array from a generator expression*

In [1]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

*Example 2-6. Cartesian product in a generator expression.*

In [2]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


## Tuples are not just immutable lists
Tuples can also be used as records with no field names.



*Example 2-7. Tuples used as records

In [1]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]
for passport in sorted(traveler_ids):
    print("%s/%s" % passport)
    
for country, _ in traveler_ids:
    print(country)

BRA/CE342567
ESP/XDA205856
USA/31195855
USA
BRA
ESP


Tuple unpacking works with any iterable object. The only requirement is that the iterable yields exactly one item per variable in the receiving tuple, unless you use a \* to capture excess items. The term *tuple unpacking* is widely used by Pythonistas.

In [8]:
# Parallel assignment
latitude, longitude = lax_coordinates # tuple unpacking
latitude, longitude

(33.9425, -118.408056)

Swapping variables can be done easily without using a temporary variable.

In [7]:
a = 1
b = 2
b, a = a, b
a, b

(2, 1)

Another example of tuple unpacking is prefixing an argument with a star when calling a function:

In [9]:
divmod(20, 8)

(2, 4)

In [10]:
t = (20, 8)
divmod(*t)

(2, 4)

In [12]:
quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

The code above also shows a further use of tuple unpacking: enabling functions to return multiple values in a way that is convenient to the caller. For example, the os.path.split() function builds a tuple (path, last_part) from a filesystem path.

In [13]:
# _ is a dummy variable; in this example we only care about storing the filename

import os
_, filename = os.path.split("/home/luciano/.ssh/idrsa.pub")
filename

'idrsa.pub'

Defining function parameters with \*args to grab arbitrary excess arguments is a classic Python feature.
In Python 3 this idea was extended to apply to parallel assignment as well.

In [14]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [16]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [18]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

The \* prefix can be applied to exactly one variable, but it can appear in any position.

*Example 2-8. Unpacking nested tuples to access the longitude.*

In [21]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386))
]

print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas:
    if longitude <= 0:
        print(fmt.format(name, latitude, longitude))

                |   lat.    |   long.  
New York-Newark |   40.8086 |  -74.0204


*Example 2-9. Defining and using a named tuple type*

In [22]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [23]:
tokyo.population

36.933

In [24]:
tokyo.coordinates

(35.689722, 139.691667)

In [26]:
tokyo[1]

'JP'

*Example 2-10. Named tuple attributes and methods (continued from Example 2-9)*

In [27]:
City._fields

('name', 'country', 'population', 'coordinates')

In [32]:
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()

OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', 21.935),
             ('coordinates', LatLong(lat=28.613889, long=77.208889))])

In [33]:
for key, value in delhi._asdict().items():
    print(key + ':', value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613889, long=77.208889)


## Slicing

### Why slices and range exclude the last item
The Pythonic convention of excluding the last item in slices and ranges works well with the zero-based indexing used in Python, C and many other languages. Some convenient features of the convention are:
* It's easy to see the length of a slice or range when only the stop positino is given: `range(3)` and `my_list[:3]` both produce three items.
* It's easy to compute the length of a slice or range when start and stop are given: just subtract `stop` - `start`. 
* It's easy to split a sequence in two parts at any index `x`, without overlapping: simply get `my_list[:x]` and `my_list[x:]`. See for example:

In [37]:
l = list(range(10, 70, 10))
l[:2]

[10, 20]

In [38]:
l[2:]

[30, 40, 50, 60]

In [40]:
l[:3]

[10, 20, 30]

In [41]:
l[3:]

[40, 50, 60]

### Slice objects
*Example 2-11. Line items from a flat file invoice*

In [44]:
invoice = """
... 0.....6.................................40........52...55........
... 1909  Pimoroni PiBrella                     $17.50    3    $52.50
... 1489  6mm Tactile Switch x20                 $4.95    2     $9.90
... 1510  Panavise Jr. - PV-201                 $28.00    1    $28.00
... 1601  PiTFT Mini Kit 320x240                $34.95    1    $34.95
"""
SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split('\n')[2:]
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])
    


    $17.50   Pimoroni PiBrella                 
     $4.95   6mm Tactile Switch x20            
    $28.00   Panavise Jr. - PV-201             
    $34.95   PiTFT Mini Kit 320x240            
 


### Multi-dimensional slicing and ellipsis
